In [231]:
#import library
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns

%matplotlib inline

In [232]:
#1. 데이터 준비
df = pd.read_csv('KRvideos_ANSI.csv',encoding = 'cp949')

df.trending_date=pd.to_datetime(df.trending_date,format='%y.%d.%m')

z = df['trending_date'].values  # trending_date 만 따로 뽑아놓기.


df.drop('publish_time', axis = 1, inplace=True)
df.drop('trending_date', axis = 1, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34567 entries, 0 to 34566
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   category_id    34567 non-null  int64
 1   views          34567 non-null  int64
 2   likes          34567 non-null  int64
 3   dislikes       34567 non-null  int64
 4   comment_count  34567 non-null  int64
dtypes: int64(5)
memory usage: 1.3 MB


In [233]:
scaler = StandardScaler()
output = scaler.fit_transform(df)

In [234]:
x = df.drop(['views'], axis=1).values # 독립변인들의 value값만 추출
y = df['views'].values # 종속변인 추출

x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장
# y도 scaling 해주기

features = ['category_id', 'likes','dislikes','comment_count']
target_features = ['views']
x_df = pd.DataFrame(x, columns=features)
x_df.head(10)


,category_id,likes,dislikes,comment_count
0,-1.668476,-0.098217,-0.038232,-0.091107
1,-1.668476,-0.096252,-0.038232,-0.090502
2,-1.668476,-0.095321,-0.038305,-0.090084
3,0.129180,-0.104112,-0.039253,-0.094129
4,0.129180,-0.093706,-0.038524,-0.091851
5,0.129180,-0.103505,-0.039253,-0.094176
6,-0.919452,-0.103198,-0.039180,-0.094083
7,-0.919452,-0.103198,-0.039107,-0.094083
8,-3.016718,-0.062181,0.003130,-0.051677
9,-3.016718,-0.060934,0.004078,-0.050282


In [235]:
y_df = pd.DataFrame(y, columns=target_features)

#  eigen-decomposition으로 보는 PCA
 


In [236]:
#1) eigen-decomposition을 이용한 PCA구하기

In [237]:
x_df.shape

(34567, 4)

In [238]:
X_cen  = x_df - x_df.mean(axis = 0)

In [239]:
X_cov = np.dot(x_df.T, X_cen) / 34566
print(X_cov)

[[ 1.00002893 -0.09797884 -0.01672524 -0.06503945]
 [-0.09797884  1.00002893  0.42804822  0.92275734]
 [-0.01672524  0.42804822  1.00002893  0.555565  ]
 [-0.06503945  0.92275734  0.555565    1.00002893]]


In [240]:
w, v = np.linalg.eig(X_cov)
print('eigenvalue :', w)
print('eigenvector :\n', v)


eigenvalue : [2.30596652 0.99647403 0.63287206 0.0648031 ]
eigenvector :
 [[ 0.08323643 -0.98923828  0.11846537 -0.02110181]
 [-0.60832602  0.01467051  0.43127376 -0.6661285 ]
 [-0.47016913 -0.13991877 -0.86144503 -0.13143892]
 [-0.63399696 -0.04018889  0.24058545  0.73386058]]


In [241]:
print('explained variance ratio :', w / w.sum())

explained variance ratio : [0.57647495 0.2491113  0.15821344 0.01620031]


# 분산 설명량 series로 보기

In [242]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(x_df)

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [243]:
data_scale = pd.DataFrame(x_df, columns=x_df.columns, index = x_df.index)
# PCA 주성분분석
pca = PCA(random_state=1107)
X_p = pca.fit_transform(data_scale)
pd.Series(np.cumsum(pca.explained_variance_ratio_))

0    0.576475
1    0.825586
2    0.983800
3    1.000000
dtype: float64

3개의 PC로 분산량의 98%이상이 설명 가능하다.

# pipeline으로 연결하기

In [244]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5)

In [245]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size = 0.3, random_state = 42)

pca = PCA(n_components=3)

#1. LinearRegression
clf = LinearRegression()
pipe = Pipeline([('pca', pca), ('linear', clf)])

#2. Lasso
lasso_reg = Lasso(alpha = 0.1, random_state =42)
pipe_lasso = Pipeline([('pca', pca), ('fit', lasso_reg)])
#3. ElasticNet
el_reg = ElasticNet(alpha = 0.1, l1_ratio = 0.5, random_state =  42)
pipe_el = Pipeline([('pca', pca), ('fit', el_reg)])
#4. Huber
hub_reg = HuberRegressor()
pipe_hub = Pipeline([('pca', pca), ('huber', hub_reg)])

# model fit
pipe.fit(X_train, y_train)
pipe_lasso.fit(X_train, y_train)
pipe_el.fit(X_train, y_train)
pipe_hub.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('huber',
                 HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True,
                                max_iter=100, tol=1e-05, warm_start=False))],
         verbose=False)

#K-Fold Validation

In [246]:
# evaluation 결과값 출력함수 정의

def print_score(model, x, y, cv1, cv2):
  score1 = cross_validate(model,x, y, cv = cv1)
  score2 = cross_validate(model, x, y, cv = cv2)
  s1 = score1['test_score']
  s2 = score2['test_score']
  m_s1 = np.mean(s1)
  m_s2 = np.mean(s2)


  print( "cv = ", cv1, "\n", m_s1, "\n")

  print("cv = ", cv2, "\n", m_s2, )

In [247]:
print("Linear_Regresson  MSE")
print_score(pipe, x_df, y_df, 3,10)

Linear_Regresson  MSE
cv =  3 
 0.6360516561178842 

cv =  10 
 0.6177189610839169


In [248]:
print("Lasso_Regresson  MSE")
print_score(pipe_lasso, x_df, y_df, 3,10)

Lasso_Regresson  MSE
cv =  3 
 0.6360516936095002 

cv =  10 
 0.6177189756059837


In [249]:
print("ElasticeNet_Regresson  MSE")
print_score(pipe_el, x_df, y_df, 3,10)

ElasticeNet_Regresson  MSE
cv =  3 
 0.6547973744477389 

cv =  10 
 0.6245434889861624


In [250]:
print("HuberRegressor  MSE")
print_score(pipe_hub, x_df, y_df, 3,10)

HuberRegressor  MSE


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

cv =  3 
 0.6417987363406701 

cv =  10 
 0.6216240609136545


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**가장 score가 높은 Elastic_Net 모델에 Test_set Fitting시키기**

In [251]:
pipe_el.fit(X_train, y_train)
pipe_el.score(X_train, y_train)

0.7769543993107896

ElasticNet의 score(R-squared)값은 0.78

In [252]:
pipe_el.fit(X_test, y_test)
pipe_el.score(X_test, y_test)

0.7151050649138441

MSE Score 71.5%까지로 향상됨.